In [ ]:
! load_ext autoreload
! autoreload 2
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import torch
import os
import json
import math
from itertools import groupby
from typing import Callable, Dict, List, Optional, Set, Tuple, Type, Union

import numpy as np
import PIL
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('../')
from lora_diffusion import LoraInjectedConv2d, LoraInjectedLinear, patch_pipe, tune_lora_scale
from lora_diffusion.lora import _find_modules, UNET_CROSSATTN_TARGET_REPLACE
from reg_lora.visual import visualize_images

os.environ["DISABLE_TELEMETRY"] = 'YES'
model_id = "runwayml/stable-diffusion-v1-5"
device = torch.device("cuda:3")
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16,local_files_only=True,revision='39593d5650112b4cc580433f6b0435385882d819').to(device)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

prompt = "style of <s1><s2>, baby lion"
torch.manual_seed(0)
# image = pipe(prompt, num_inference_steps=50, guidance_scale=7).images[0]

# image  # nice. diffusers are cool.

In [ ]:
from reg_lora.clip_reg import CLIPTiDataset, CLIPTiScoreCalculator
from lora_diffusion.lora import patch_pipe
lora_ckpt = '../outputs/checkpoints/output_dog_Ti-clip_Nonorm_3e-5/lora_weight_e6_s1000.safetensors'
torch.manual_seed(0)

patch_pipe(
    pipe,
    lora_ckpt,
    patch_text=False,
    patch_ti=True,
    patch_unet=True,
    filter_crossattn_str = 'cross+self'
)

data  = CLIPTiDataset(instance_data_root="../custom_data/data/dog",
                      placeholder_tokens="<krk1>",
                      class_token='dog'
                      )
clip = CLIPTiScoreCalculator(text_model = pipe.text_encoder.text_model,
                             tokenizer = pipe.tokenizer,
                             placeholder_tokens = "<krk1>",
                             class_token_len = 1)
data_loader = torch.utils.data.DataLoader(data, batch_size=2, shuffle=False, num_workers=1)
clip_batch = next(iter(data_loader))
instance_texts = clip_batch['text']
instance_images = [image for image in clip_batch['np_instance_image']]
outputs = clip.clip_forward(instance_texts, instance_images)



In [ ]:
lora_path = '/home/zhicai/poseVideo/lora-master/output_dog_Ti-Noclip_norm/lora_weight_e62_s10000.safetensors'
# lora_path = '/home/zhicai/poseVideo/lora-master/output_dog_Ti-Noclip_norm/lora_weight_e62_s10000.safetensors'
# lora_path = '../output_dog_cross+self_tR0.001_nR0.001/lora_weight_e12_s2000.safetensors'

patch_pipe(
    pipe,
    lora_path,
    patch_text=False,
    patch_ti=True,
    patch_unet=True,
    filter_crossattn_str = 'cross+self'
)

prompts = 'photo of a <krk1> dog swimming in the pool' 
tune_lora_scale(pipe.unet, 0)
torch.manual_seed(0)
img = pipe([prompts] * 4, num_inference_steps=50, guidance_scale=7).images
visualize_images(img,outpath=' ',nrow = 5, save = False)

In [ ]:
import copy
import glob
import re
# '../output_dog_cross+self_tR0.001_nR0.001'
_lora_path = '../output/output_dog_Ti-clip_norm_3e-5'
prompts = ['photo of a <krk1> dog swimmking in the pool'] 
images = []
bs = 4
cnt = 0
_files = os.listdir(_lora_path)
pattern = r'lora_weight_e\d+_s\d{3,}.safetensors'
files = [f for f in _files if re.match(pattern, f)]

for i, path in enumerate(sorted(files, key = lambda x: int(re.match(r'.*e([0-9]+).*', x.split('/')[-1])[1]), reverse=False)):
    if i % 10 ==0:
        cnt +=1 
        lora_path = os.path.join(_lora_path, path)
        print(lora_path)
        pipe_copy = copy.deepcopy(
            pipe)
        torch.manual_seed(0)
        patch_pipe(
            pipe_copy,
            lora_path,
            patch_text=False,
            patch_ti=True,
            patch_unet=True,
            filter_crossattn_str = 'cross+self'
        )
        # pipe.unet
        tune_lora_scale(pipe_copy.unet, 0.5)
        # tune_lora_scale(pipe_copy.text_encoder, 1)
        for prompt in prompts:
            prompt = [prompt]*bs
            img = pipe_copy(prompt = prompt, num_inference_steps=50, guidance_scale=6).images
            images.extend(img)
if len(images) > 0:
    visualize_images(images,outpath='photo_of_krk1_norm_dog', nrow=4, show=False,save=True)
else:
    print('no images')


In [ ]:
import copy
import glob
import re
# '../output_dog_cross+self_tR0.001_nR0.001'
_lora_path = '../output_dog_cross+self_tR0.001_Scale-nR0.01'
prompts = ['<krk> dog in a construction outfit','<krk> dog in times square'] 
images = []
bs = 2
cnt = 0
_files = os.listdir(_lora_path)
pattern = r'lora_weight_e\d+_s\d{4,}.safetensors'
files = [f for f in _files if re.match(pattern, f)]

for path in sorted(files, key = lambda x: int(re.match(r'.*e([0-9]+).*', x.split('/')[-1])[1]), reverse=True):
    cnt +=1 
    lora_path = os.path.join(_lora_path, path)
    print(lora_path)
    pipe_copy = copy.deepcopy(
        pipe)
    torch.manual_seed(0)
    patch_pipe(
        pipe_copy,
        lora_path,
        patch_text=False,
        patch_ti=True,
        patch_unet=True,
        filter_crossattn_str = 'cross+self'
    )
    # pipe.unet
    tune_lora_scale(pipe_copy.unet, 1)
    # tune_lora_scale(pipe_copy.text_encoder, 1)
    for prompt in prompts:
        prompt = [prompt]*bs
        img = pipe_copy(prompt = prompt, num_inference_steps=50, guidance_scale=6).images
        images.extend(img)
    if cnt >=3 :
        break
if len(images) > 0:
    visualize_images(images,outpath='photo_of_krk_dog.jpg', nrow=4, save=False)
else:
    print('no images')


In [ ]:
import torch
import torch.nn as nn

# 创建一个简单的神经网络模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.layer1 = nn.Linear(2, 4)
        self.layer2 = nn.Linear(4, 3)
        self.layer3 = nn.Linear(3, 1)

        # 将第二层的参数设置为不需要梯度
        for param in self.layer2.parameters():
            param.requires_grad = False

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# 创建输入数据
input_data = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
input_data2 = torch.tensor([[2.0, 2.0], [3.0, 4.0]])

# 创建模型实例
model = SimpleModel()


# 进行前向传播
model.zero_grad()
output = model(input_data)
output2 = model(input_data2)

# 创建损失函数和优化器
criterion = nn.MSELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# 计算损失并进行反向传播
loss = criterion(output2, torch.tensor([[0.0], [1.0]]))
loss.backward(retain_graph=True)

# 打印第一层和第三层的参数的梯度
print("Layer 1 gradients:")
print(model.layer1.weight.grad)
print(model.layer1.bias.grad)

print("\nLayer 2 gradients (should be None):")
print(model.layer2.weight.grad)
print(model.layer2.bias.grad)

print("\nLayer 3 gradients:")
print(model.layer3.weight.grad)
print(model.layer3.bias.grad)


In [ ]:
from torch.autograd import grad
#             retain_graph=(i < len(loglikelihoods) - 1)

model.zero_grad()
output2 = model(input_data2)
grad_out = grad((2*output2).mean(), model.layer2.parameters(), retain_graph=True)


In [ ]:
import torch
import torch.nn as nn
from accelerate import Accelerator

accelerator = Accelerator()

class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.layer1 = nn.Linear(2, 4)
        self.layer2 = nn.Linear(4, 3)
        self.layer3 = nn.Linear(3, 1)

        # 将第二层的参数设置为不需要梯度
        for param in self.layer2.parameters():
            param.requires_grad = False

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# 创建输入数据
input_data = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
input_data2 = torch.tensor([[2.0, 2.0], [3.0, 4.0]])

# 创建模型实例
model = SimpleModel()
model, criterion = accelerator.prepare(model, nn.MSELoss())
input_data, input_data2 = accelerator.prepare(input_data, input_data2)


In [ ]:
# 进行前向传播
model.zero_grad()
output = model(input_data)
output2 = model(input_data2)

# 计算损失并进行反向传播
loss = criterion(output2, torch.tensor([[0.0], [1.0]]).to(accelerator.device))
accelerator.backward(loss, retain_graph=True)


In [ ]:
# 打印第一层和第三层的参数的梯度
print("Layer 1 gradients:")
print(accelerator.grad(model.layer1.weight))
print(accelerator.grad(model.layer1.bias))

print("\nLayer 2 gradients (should be None):")
print(accelerator.grad(model.layer2.weight))
print(accelerator.grad(model.layer2.bias))

print("\nLayer 3 gradients:")
print(accelerator.grad(model.layer3.weight))
print(accelerator.grad(model.layer3.bias))


In [1]:
import torch
import torch.nn as nn
from accelerate import Accelerator, notebook_launcher
from accelerate.utils import reduce , broadcast

class LoraInjectedLinear(nn.Module):
    def __init__(
        self, in_features, out_features, bias=False, r=4, dropout_p=0.1, scale=1.0
    ):
        super().__init__()

        if r > min(in_features, out_features):
            raise ValueError(
                f"LoRA rank {r} must be less or equal than {min(in_features, out_features)}"
            )
        self.r = r
        self.linear = nn.Linear(in_features, out_features, bias)
        self.lora_down = nn.Linear(in_features, r, bias=False)
        self.dropout = nn.Dropout(dropout_p)
        self.lora_up = nn.Linear(r, out_features, bias=False)
        self.scale = scale
        self.selector = nn.Identity()

        nn.init.normal_(self.lora_down.weight, std=1 / r)
        nn.init.zeros_(self.lora_up.weight)

    def forward(self, input):

        return (
            self.linear(input)
            + self.dropout(self.lora_up(self.selector(self.lora_down(input))))
            * self.scale
        )

    def realize_as_lora(self):
        return self.lora_up.weight.data * self.scale, self.lora_down.weight.data
   
    def get_reg_loss(self,reg_vector = None):
        if reg_vector is None:
            if getattr(self, 'fisher'):
                weight_reg = (self.lora_up.weight.pow(2) * self.fisher).sum()
            else: 
                # L2 norm of the weight matrix
                weight_reg = torch.norm(self.lora_up.weight, dim = [1], p = 2) + \
                         torch.norm(self.lora_down.weight, dim = [0], p = 2)
            return weight_reg
        else:
            lora_project_vector = self.lora_up(self.lora_down(reg_vector))
            return torch.norm(lora_project_vector, dim =[1,2], p = 2)
        
    def update_fisher(self):
        # update fisher information of lora-up matrix
        # TODO
        if not getattr(self,'fisher'):
            self._fisher_steps = 1
            setattr(self,'fisher',torch.zeros_like(self.lora_up.weight))
        else:
            self._fisher_steps += 1
            _a = self._fisher_steps
            self.fisher = self.fisher*((_a-1)/_a) + self.lora_up.weight.grad.detach().pow(2)*(1/_a)

    def set_selector_from_diag(self, diag: torch.Tensor):
        # diag is a 1D tensor of size (r,)
        assert diag.shape == (self.r,)
        self.selector = nn.Linear(self.r, self.r, bias=False)
        self.selector.weight.data = torch.diag(diag)
        self.selector.weight.data = self.selector.weight.data.to(
            self.lora_up.weight.device
        ).to(self.lora_up.weight.dtype)

def train():
    model = LoraInjectedLinear(20,40,r=4)
    accelerator = Accelerator()
    model = accelerator.prepare(model)
    ewc_prams = []
    freeze_prams = []
    for name, parameters in model.named_parameters():
        if 'lora_up' in name:
            ewc_prams.append(parameters)
        else:
            freeze_prams.append(parameters)
    with open('log.txt','w') as f:
    # for _ in range(10):
        in_features = torch.randn(1, 3, 10, 20).to(accelerator.device)
        out_features = model(in_features)
        likelihood = out_features.mean()
        accelerator.backward(likelihood, retain_graph=False)
        model.update_fisher()
        print(model.fisher)
        f.write(f"current process {accelerator.process_index}, fisher matrix {model.fisher}")
        reduce(model.fisher, reduction = 'mean')
        accelerator.log(f"current process {accelerator.process_index}, fisher matrix {model.fisher}")
        broadcast(model.fisher, from_prcocess = 0)
        accelerator.log(f"current process {accelerator.process_index}, fisher matrix {model.fisher}")


        # accelerator.backward(likelihood, retain_graph=False)
notebook_launcher(train, num_processes=2, use_port=28700)


2023-08-11 10:32:42.798609: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Launching training on 2 GPUs.


In [8]:
model

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_3661474/2355735933.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3661474/2355735933.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [25]:
dir(accelerator)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_custom_objects',
 '_dataloaders',
 '_do_sync',
 '_get_devices',
 '_get_named_parameters',
 '_load_model_state_pre_hook',
 '_models',
 '_optimizers',
 '_prepare_deepspeed',
 '_prepare_fsdp',
 '_prepare_ipex',
 '_prepare_megatron_lm',
 '_prepare_one',
 '_save_model_state_pre_hook',
 '_schedulers',
 'accumulate',
 'autocast',
 'backward',
 'clear',
 'clip_grad_norm_',
 'clip_grad_value_',
 'ddp_handler',
 'device',
 'device_placement',
 'dispatch_batches',
 'distributed_type',
 'end_training',
 'even_batches',
 'fp8_recipe_handler',
 'free_memory',
 'gather',
 'gather_for_metrics',
 'get_state_dict',
 'get_tracker',
 'grad

In [40]:
import multiprocessing
import time
import random

def init_worker():
    print(f"Worker {multiprocessing.current_process().name} initializing")

def worker_function(task_id):
    print(f"Worker {multiprocessing.current_process().name} executing task {task_id}")
    time.sleep(random.randint(4,5))
    return task_id * 2

if __name__ == "__main__":
    num_tasks = 10
    processes = 2  # 使用默认工作进程数量

    # 创建进程池并设置参数
    with multiprocessing.Pool(processes=processes, initializer=init_worker, maxtasksperchild=1) as pool:
        tasks = list(range(num_tasks))

        # 调用 worker_function 并获取结果
        results = pool.map(worker_function, tasks)

    print("Results:", results)


Worker ForkPoolWorker-140 initializing
Worker ForkPoolWorker-140 executing task 0
Worker ForkPoolWorker-141 initializing
Worker ForkPoolWorker-141 executing task 2
Worker ForkPoolWorker-141 executing task 3
Worker ForkPoolWorker-140 executing task 1
Worker ForkPoolWorker-142 initializing
Worker ForkPoolWorker-142 executing task 4
Worker ForkPoolWorker-143 initializing
Worker ForkPoolWorker-143 executing task 6
Worker ForkPoolWorker-142 executing task 5
Worker ForkPoolWorker-143 executing task 7
Worker ForkPoolWorker-144 initializing
Worker ForkPoolWorker-144 executing task 8
Worker ForkPoolWorker-145 initializing
Worker ForkPoolWorker-144 executing task 9
Results: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [37]:
import multiprocessing

def worker_function(task_id):
    print(f"Worker {multiprocessing.current_process().name} executing task {task_id}")
    time.sleep(random.randint(1, 5))
    return task_id * 2

if __name__ == "__main__":
    num_tasks = 10
    processes = 2

    # 创建进程池
    with multiprocessing.Pool(initializer=init_worker,processes=processes,maxtasksperchild=2) as pool:
        tasks = list(range(num_tasks))
        results = []

        # 同步执行任务并获取结果
        for task_id in tasks:
            result = pool.apply(worker_function, args=(task_id,))
            results.append(result)

    print("Results:", results)


Worker ForkPoolWorker-130 initializing
Worker ForkPoolWorker-130 executing task 0
Worker ForkPoolWorker-131 initializing
Worker ForkPoolWorker-131 executing task 1
Worker ForkPoolWorker-130 executing task 2
Worker ForkPoolWorker-131 executing task 3
Worker ForkPoolWorker-132 initializing
Worker ForkPoolWorker-132 executing task 4
Worker ForkPoolWorker-133 initializing
Worker ForkPoolWorker-133 executing task 5
Worker ForkPoolWorker-132 executing task 6
Worker ForkPoolWorker-133 executing task 7
Worker ForkPoolWorker-134 initializing
Worker ForkPoolWorker-134 executing task 8
Worker ForkPoolWorker-135 initializing
Worker ForkPoolWorker-135 executing task 9
Results: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [17]:
import re
num = []
with open('../log.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        outs = re.search('(\+|-)\d+',line)[0]
        if outs is not None:
            print(outs)
            # num.append(int(outs[0]))
        else:
            raise ValueError('No number found')


-30
-400
-600
-196
-66
-191
-147
-26
-89
+604
+311
+69
+216
+63
+254
+93
+29
+201


In [13]:
import glob 
import os
ckpt_lists = glob.glob(os.path.dirname(os.getcwd()) + '/outputs/checkpoints/*/shareTI_text_0.01_ewc_0.01/decay_lr_5e-4_gas_4/lora_weight_s2000.safetensors')
len(ckpt_lists)

18

In [12]:
ckpt_lists = glob.glob(os.path.dirname(os.getcwd()) + '/outputs/checkpoints/*/shareTI_baseline/decay_lr_5e-4_gas_4/lora_weight_s2000.safetensors')
len(ckpt_lists)

37

In [4]:
import glob 
import os
ckpt_lists = glob.glob(os.path.dirname(os.getcwd()) + '/outputs/checkpoints/*/shareTI_text_0.01_freeze-down/decay_lr_5e-4_gas_4/lora_weight_s2000.safetensors')
len(ckpt_lists)
os.makedirs('/data/zhicai/code/Text-regularized-customization/outputs/sample/aaaa/shareTI_text_0.01_ewc_0.01/decay_lr_5e-4_gas_4/lora_weight_s1600',exist_ok=True)

In [5]:
cnt

0